Делаем распознавание лиц

1. Датасет [CelebA](http://mmlab.ie.cuhk.edu.hk/projects/CelebA.html)
2. Можно использовать претренированные сети
3. Пробовать на другом выравнивании?

# Знакомимся с CelebA

Качаем датасет, удостоверяемся, что понимаем как он устроен

- вывести пару примеров
- проверить, что все размеры одинаковые
- посмотреть на выравнивание (переведите в grayscale и посчитайте среднее лицо -- это даст представление о выравнивании)

In [73]:
%matplotlib inline
import matplotlib.pyplot as plt
import cv2
import numpy as np
from pathlib import Path
import torch
import torch.nn as nn
import torch.nn.functional as F

import torchvision
from torch.utils.data import DataLoader
from tqdm.auto import tqdm


def load(path):
    mean = np.array([0.485, 0.456, 0.406]).reshape([1, 1, 3])
    std = np.array([0.229, 0.224, 0.225]).reshape([1, 1, 3])
    img = cv2.imread(path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img.astype(np.float32) / 255.0
    img = (img - mean) / std
    img = img.transpose([2, 0, 1]).astype(np.float32)
    return img


class CelebA:
    def __init__(self, root, train=True, transform=None):
        root = Path(root)
        data = []
        
        with open(root / "list_eval_partition.txt") as A, open(root / "identity_CelebA.txt") as B:
            for a, b in zip(A, B):
                name, split = a.strip().split(" ")
                _, label = b.strip().split(" ")
                split, label = int(split), int(label)
                
                if (split == 1 and train) or (split == 0 and not train):
                    data.append(
                        dict(
                            path=str(root / "../img_align_celeba" / name),
                            label=int(label),
                        )
                    )
        self.data = data
        self.transform = transform
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, item):
        path = self.data[item]['path']
        img = load(path)
        return dict(
            img=img,
            label=self.data[item]['label']
        )
    
    
    
ds = CelebA("./data/celeba")

In [56]:
len(ds)



19867

In [61]:
from collections import Counter
cnt = Counter()

for x in tqdm(ds):
    cnt.update([x['label']])

In [63]:
from torchvision.models import resnet50

In [68]:
net = resnet50(pretrained=True)
net.fc = nn.Identity()
net.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [77]:
dl = DataLoader(ds, shuffle=False, batch_size=4, num_workers=4)
descriptors = []
labels = []
with torch.no_grad():
    for batch in tqdm(dl):
        output = net(batch['img']).cpu().numpy()
        descriptors.append(output)
        labels.append(batch['label'].cpu().numpy())
        
X = np.concatenate(descriptors, dim=0)
y = np.concatenate(labels, dim=0)

KeyboardInterrupt: 

In [79]:
X = np.concatenate(descriptors, axis=0)
y = np.concatenate(labels, axis=0)

In [80]:
X

array([[1.3057031e-02, 9.6057540e-01, 1.0012240e+00, ..., 6.3794434e-02,
        3.0139231e-04, 6.7741878e-02],
       [1.5929116e-01, 1.2700621e+00, 1.9592046e+00, ..., 1.2924200e-01,
        2.3390128e-01, 3.1160673e-01],
       [1.3679238e-01, 7.4820668e-01, 1.5299295e+00, ..., 0.0000000e+00,
        7.1595594e-02, 2.7225909e-01],
       ...,
       [3.1347152e-02, 5.7980090e-01, 1.0299991e+00, ..., 1.4434235e-01,
        1.3754341e-01, 2.1105382e-01],
       [1.5333715e-01, 3.7051988e-01, 1.7522386e+00, ..., 1.5532699e-02,
        7.5858019e-02, 8.3124071e-02],
       [1.2640564e-01, 1.2454096e+00, 2.4079039e+00, ..., 2.3998079e-01,
        8.9403823e-02, 1.3036683e-01]], dtype=float32)

# Делаем поиск лиц!

- берем и прогоняем через предобученную сеть все картинки -> получаем вектора, складываем их в массив, дальше работать будем именно с ними
- пробуем без обучения сделать поиск лиц (надо взять не-тренировочный сплит и убедиться что там другие метки)
- на векторах искать 5-ближайших -> их метки -- это ответ
- посчитать метрику (top1, top5, )


- Учим модельку поверх векторов с contrastive loss'ом
- Замеряем что получается